In [ ]:
import hashlib

def compute_hash(path, algorithm):
    h = hashlib.new(algorithm)
    with open(path, "rb") as f:
        h.update(f.read())
    return h.hexdigest()

sample = r"C:\Users\tomek\Downloads\ProcessMonitor\Procmon.exe"

print("MD5: ", compute_hash(sample, "md5"))
print("SHA1: ", compute_hash(sample, "sha1"))
print("SHA256:", compute_hash(sample, "sha256"))

MD5:  c3e77b6959cc68baee9825c84dc41d9c
SHA1:  bc18a67ad4057dd36f896a4d411b8fc5b06e5b2f
SHA256: 3b7ea4318c3c1508701102cf966f650e04f28d29938f85d74ec0ec2528657b6e


Changing even just one byte in the file (can use a text editor) causes the hash to change completely (avelanche effect)

In [ ]:
import re

def extract_strings(path):
    with open(path, "rb") as f:
        data = f.read()
    pattern = rb"[ -~]{4,}"
    return re.findall(pattern, data)

strings = extract_strings(sample)

for s in strings[:20]:
    print("Extracted string: ", s.decode(errors="ignore")) #ignores errors

Extracted string:  !This program cannot be run in DOS mode.
Extracted string:  V*0T
Extracted string:  0RichU
Extracted string:  .text
Extracted string:  `.rdata
Extracted string:  @.data
Extracted string:  .rsrc
Extracted string:  @.reloc
Extracted string:  hpqQ
Extracted string:  h`EN
Extracted string:  h|nN
Extracted string:  h\nN
Extracted string:  hlnN
Extracted string:  =UUU
Extracted string:  h_rM
Extracted string:  hDLN
Extracted string:  h`GO
Extracted string:  hDLN
Extracted string:  h|GO
Extracted string:  hDLN


In [8]:
!pip install pefile

  Using cached pefile-2024.8.26-py3-none-any.whl.metadata (1.4 kB)
Using cached pefile-2024.8.26-py3-none-any.whl (74 kB)


In [ ]:
import pefile

pe = pefile.PE(sample)

# Entry Point: where execution begins
# Image Base: preferred load address in memory
# Import table: all external functions (APIs) the binary relies on

print("Entry Point:", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint))
print("Image Base:", hex(pe.OPTIONAL_HEADER.ImageBase))

print("\nImported DLLs and functions:")
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(" ", entry.dll.decode())
    for imp in entry.imports[:5]:
        print(" -", imp.name.decode() if imp.name else "None")

Entry Point: 0xa7f70
Image Base: 0x400000

Imported DLLs and functions:
  WS2_32.dll
 - getsockname
 - listen
 - recv
 - closesocket
 - socket
  VERSION.dll
 - GetFileVersionInfoW
 - VerQueryValueW
 - GetFileVersionInfoSizeW
  COMCTL32.dll
 - ImageList_ReplaceIcon
 - ImageList_SetBkColor
 - ImageList_AddMasked
 - ImageList_BeginDrag
 - ImageList_EndDrag
  FLTLIB.DLL
 - FilterSendMessage
 - FilterGetMessage
 - FilterReplyMessage
 - FilterConnectCommunicationPort
  KERNEL32.dll
 - AcquireSRWLockExclusive
 - AcquireSRWLockShared
 - InitializeSRWLock
 - GetSystemInfo
 - VerSetConditionMask
  USER32.dll
 - LoadStringA
 - DrawEdge
 - GetMessageW
 - TranslateMessage
 - DispatchMessageW
  GDI32.dll
 - SaveDC
 - RestoreDC
 - SetBrushOrgEx
 - SetPixel
 - PatBlt
  COMDLG32.dll
 - ChooseColorW
 - GetOpenFileNameW
 - PrintDlgW
 - ChooseFontW
 - FindTextW
  ADVAPI32.dll
 - RegQueryValueExW
 - ConvertStringSidToSidW
 - ConvertSidToStringSidW
 - RegSetValueW
 - RegEnumKeyW
  SHELL32.dll
 - SHGetSpecia

Procmon imports many legitimate Windows APIs, such as:
• kernel32.dll
• user32.dll
• advapi32.dll
If this were malware, suspicious API imports might include:
• CreateRemoteThread (process injection)
• VirtualAllocEx (shellcode allocation)
• GetProcAddress and LoadLibraryA (dynamic API resolving)
• WinExec or ShellExecuteA (execution of child processes)

In [11]:
!pip install yara-python

  Using cached yara_python-4.5.4-cp312-cp312-win_amd64.whl.metadata (3.0 kB)
Using cached yara_python-4.5.4-cp312-cp312-win_amd64.whl (1.8 MB)


In [13]:
import yara

rule_source = """
rule ContainsHTTP {
    strings:
        $s = "http"
    condition:
        $s
}
"""
rules = yara.compile(source=rule_source)
matches = rules.match(sample)
print(matches)

AttributeError: module 'yara' has no attribute 'compile'

try other softwares, look for anything interesting, anything suspicious. 